In [2]:
import pandas as pd
from konlpy.tag import Okt
import os
import glob
import re 

okt = Okt()
current_dir = os.getcwd()

file_path = '../../data/crawling/cafelist/cafelist_연남동.csv'
data = pd.read_csv(file_path)
print(data)
stopwords = data['name'].tolist()
split_stopwords = []

for word in stopwords:
    split_stopwords.extend(word.split())
    
print(current_dir)
input_directory = os.path.join('../../data', 'crawling', 'blog_review', 'blog_review_result_list', '연남동')
output_directory = os.path.join('../../data', 'RE_konlpy', 'blog_reviews', 'yeonnam')
filepaths = glob.glob(input_directory + '/*.csv')

for filepath in filepaths:
    try:
        data = pd.read_csv(filepath)
    except pd.errors.EmptyDataError:
        print(f'Skipped empty file: {filepath}')
        continue
    
    for stopword in split_stopwords:
        for column in data.columns:
            if column != 'title':
                pattern = re.compile(stopword, re.IGNORECASE)  
                data[column] = data[column].apply(lambda x: pattern.sub('', str(x)))
    
    for column in data.columns:
        data[column] = data[column].apply(lambda x: [okt.normalize(word[0]) for word in okt.pos(' '.join(okt.morphs(x))) if word[1] in ['Noun', 'Adjective']])
        data[column] = data[column].apply(lambda x: [okt.morphs(word, stem=True)[0] if okt.pos(word)[0][1] == 'Adjective' else word for word in x])
    
    base = os.path.basename(filepath)
    new_filepath = os.path.join(output_directory, f'{os.path.splitext(base)[0]}_stopwords.txt')
    data.to_csv(new_filepath, index=False, sep='\t', header=None)

print("All files have been processed.")

     Unnamed: 0        name cafe_type
0             1        로컬코드       브런치
1             2  브레디포스트 연남점    카페,디저트
2             3        팽페르뒤    카페,디저트
3             4       코코로카라      베이커리
4             5         터틀힙    카페,디저트
..          ...         ...       ...
296         299        크리멜트      베이커리
297         300        앤스커피    카페,디저트
298         301     로앤쿡드 연남       브런치
299         302        요거로밀    카페,디저트
300         303        라보아르    카페,디저트

[301 rows x 3 columns]
C:\Users\kim\Desktop\2023-2\빅데이터프로그래밍\cabojago\code\konlpy
All files have been processed.
